In [4]:
import os
from dotenv import load_dotenv
import chromadb
from chromadb.config import Settings

load_dotenv(".env")

CHROMA_SERVER_AUTH_CREDENTIALS = os.getenv("CHROMA_SERVER_AUTH_CREDENTIALS")

client = chromadb.HttpClient(
    host="https://chroma.liara.run",
    settings=Settings(
        chroma_client_auth_provider="chromadb.auth.token_authn.TokenAuthClientProvider",
        chroma_client_auth_credentials=CHROMA_SERVER_AUTH_CREDENTIALS,
    ),
)

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(".env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL")


openai_client = OpenAI(api_key=OPENAI_API_KEY, base_url=OPENAI_BASE_URL)

embedding = (
    openai_client.embeddings.create(
        input="رونالدو", model="text-embedding-3-small"
    )
    .data[0]
    .embedding
)

print(embedding)

[-0.006527856457978487, -0.010205594822764397, -0.01690414734184742, -0.013800571672618389, -0.004937273450195789, -0.02710457146167755, -0.0029458117205649614, 0.056071288883686066, -0.029380526393651962, -0.04013959318399429, -0.044360458850860596, -0.00201603164896369, -0.037367064505815506, -0.0019539601635187864, 0.025801068171858788, -0.00206258543767035, -0.05069175362586975, 0.02058705873787403, 0.018176615238189697, -0.0044717369601130486, 0.047670938074588776, 0.02522173337638378, 0.006564064882695675, 0.006264052353799343, 0.04040857031941414, -0.004414837807416916, 0.019521497189998627, -0.017100708559155464, -0.030125385150313377, 0.009383147582411766, 0.01745244674384594, -0.04378112405538559, 0.029277075082063675, -0.011348745785653591, 0.05367118865251541, 0.016252396628260612, -0.05470571294426918, 0.013190201483666897, -0.007350304163992405, 0.016800696030259132, 0.01940770074725151, -0.01261086668819189, 0.0065588923171162605, -0.011938425712287426, 0.014566120691597

In [ ]:
client.create_collection(name="products_openai")

In [ ]:
import pandas as pd

sample_products = pd.read_csv("../../data/sample/products.csv")
product_collection = client.get_collection(name="products_openai")

documents = []
ids = []
metadatas = []
embeddings = []

for i, row in sample_products.iterrows():
    print(i, row["title_fa"])
    embedding = (
        openai_client.embeddings.create(
            input=row["title_fa"], model="text-embedding-3-small"
        )
        .data[0]
        .embedding
    )
    ids.append(str(row["id"]))
    documents.append(row["title_fa"])
    embeddings.append(embedding)
    metadatas.append(row.to_dict())

In [ ]:
product_collection = client.get_collection(name="products_openai")

product_collection.upsert(
    documents=documents,
    metadatas=metadatas,
    ids=ids,
    embeddings=embeddings,
)

print(product_collection.peek(limit=5))